# Hydrogen Molecule Simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ, Aer

from H2_helpers import *

## Exact Theoretical Approach

Here we find the exact bond energy of the H2 molecule at the bond length of 0.735 Angstroms or 7.35x10$^{-2}$ nm, where the molecule is most stable.

In [ ]:
bond_length= 0.735
qubit_op, operator= get_H2_Hamiltonian(bond_length)
result = NumPyMinimumEigensolver(qubit_op).run()
exact_result = operator.process_algorithm_result(result)

print('========= Ground State Energy =========')
print('Total Energy (Hartree): {:.10f}'.format(exact_result.energy))
print('Electronic Energy (Hartree): {:.10f}'.format(exact_result.electronic_energy))
print('Nuclear Repulsion Energy (Hartree): {:.10f}'.format(exact_result.nuclear_repulsion_energy))

Now lets find the bond energy of the H2 molecule at different bond lengths. Plot the total energy vs. bond length. Also compare how the electronic energy changes with how the nuclear repulsion energy changes with bond length

In [ ]:
bond_lengths= sorted(list(np.arange(0.2, 4.1, 0.1))+[0.735])
theory_energies=[]
theory_electronic_energies=[]
theory_nuclear_repulsion=[]



## Variational Quantum Eigensolver (VQE) approach

Let's use a hybrid quantum-classical algorithm to find the bond energy of H2.

### Noiseless

Here, we use a variational quantum algorithm to find the energy of the H2 molecule at it's most stable bond length (0.735 A) using a noiseless quantum simulator.

We start by using an RYRZ variational circuit: https://qiskit.org/documentation/stubs/qiskit.aqua.components.variational_forms.RYRZ.html
with depth 5.

We optimize our quantum circuit using a Simultaneous Perturbation Stochastic Approximation (SPSA) optimizer: https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.SPSA.html

In [ ]:
bond_length= 0.735
qubit_op, operator= get_H2_Hamiltonian(bond_length)

var_form = RYRZ(qubit_op.num_qubits, depth=5, entanglement='full')
optimizer = SPSA()
simulator = Aer.get_backend('qasm_simulator')
algo = VQE(qubit_op, var_form, optimizer)
result = algo.run(QuantumInstance(simulator))

result = operator.process_algorithm_result(result)
print('========= Ground State Energy =========')
print('Total Energy (Hartree): {:.10f}'.format(result.energy))
print('Electronic Energy (Hartree): {:.10f}'.format(result.electronic_energy))
print('Nuclear Repulsion Energy (Hartree): {:.10f}'.format(result.nuclear_repulsion_energy))

Now let's explore what happens if we vary the variational circuit depth. Plot the output energy of VQE algorithm vs. circuit depth. Can you explain your observation? 

In [ ]:
circuit_depths=np.linspace(1,10,11)



Next, find the energy of the H2 molecule for a few different bond lengths using VQE and plot them against the exact solutions computed earlier.

In [ ]:
ideal_vqe_energies= []
vqe_bond_lengths= bond_lengths[::2]
depth=1 #Change



### Noisy simulation

The quantum computers that we have access to today are noisy systems, and the noise could change the output of our quantum algorithm. In this section we will add a noise component to our quantum simulation. You can find more information about different noise models here: https://qiskit.org/documentation/apidoc/aer_noise.html

In [ ]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_vigo') #Here we chose the noise model that the ibmq_virgo device experiences
noise_model = NoiseModel.from_backend(backend)

In [ ]:
bond_length= 0.735
qubit_op, operator= get_H2_Hamiltonian(bond_length)

var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='full')
optimizer = SPSA()
simulator = Aer.get_backend('qasm_simulator')
algo = VQE(qubit_op, var_form, optimizer)
result = algo.run(QuantumInstance(simulator), NoiseModel=noise_model) # We add the noise model here

result = operator.process_algorithm_result(result)
print('========= Ground State Energy =========')
print('Total Energy (Hartree): {:.10f}'.format(result.energy))
print('Electronic Energy (Hartree): {:.10f}'.format(result.electronic_energy))
print('Nuclear Repulsion Energy (Hartree): {:.10f}'.format(result.nuclear_repulsion_energy))

Now check how the quantum algorithm performance changes by varying the depth. Plot the calculated energy values when varying the algorithm depth and choose the best depth. Note that we need the energy to be accurate to 0.001 Hartree for a successful simulation. 

In [ ]:
depths=np.linspace(2,10,5)

plt.fill_between(depths, exact_result.energy-0.001, exact_result.energy+0.001, color='green')
plt.ylabel('Energy (Hartree)')
plt.xlabel('Circuit depth')
plt.show()

Next, find the energy of the H2 molecule for a few different bond lengths using VQE with a noise model of a real quantum hardware and plot them against the exact solutions computed earlier.

In [ ]:
noisy_vqe_energies= []
noisy_vqe_bond_lengths= vqe_bond_lengths[:12]
depth=1 #Change



### Quantum device

In [ ]:
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_london')
noise_model = NoiseModel.from_backend(backend)

First, let's train our circuit using a simulator

In [ ]:
depth=1 #Change

bond_length= 0.735
qubit_op, operator= get_H2_Hamiltonian(bond_length)

var_form = RYRZ(qubit_op.num_qubits, depth=depth, entanglement='full')
optimizer = SPSA()
simulator = Aer.get_backend('qasm_simulator')
algo = VQE(qubit_op, var_form, optimizer)
simulation_result = algo.run(QuantumInstance(simulator), NoiseModel=noise_model)

Now, let's run the optimal circuit on an actual quantum computer!

In [ ]:
optimal_point= simulation_result['optimal_point']
algo = VQE(qubit_op, var_form, COBYLA(1),initial_point=optimal_point)
hardware_result = algo.run(backend)#, shots=8192)
hardware_result = operator.process_algorithm_result(hardware_result)

print('========= Ground State Energy =========')
print('Total Energy (Hartree): {:.10f}'.format(hardware_result.energy))
print('Electronic Energy (Hartree): {:.10f}'.format(hardware_result.electronic_energy))
print('Nuclear Repulsion Energy (Hartree): {:.10f}'.format(hardware_result.nuclear_repulsion_energy))